In [318]:
class PIDController():

    AUTO_MODE = 1
    OFF_MODE = 0

    def __init__(self, pgain=1, itime=1, dtime=0, auto_max=100,
                 auto_min=0, do_ifactor=False):

        self.auto_min = auto_min
        self.auto_max = auto_max
        self.pgain = pgain
        self.itime = itime
        self.oneoveritime = 1 / self.itime  # used to calc accumulated_error time
        self.dtime = dtime
        self.dif = do_ifactor

        self.accumulated_error = 0
        self.bump = 0
        self.last_output = 0
        self.last_error = 0
        self.last_pv = 0

    def off_to_auto(self, pv, sp):
        """
        Calculate bump for off-to-auto transfer
        :param pv: current process temp to use for bumpless xfer calculation
        """
        self.man_to_auto(pv, sp, 0)

    def man_to_auto(self, pv, sp, op):
        err = sp - pv
        uk0 = self.pgain * err
        self.bump = op - uk0
        
    set_bump = man_to_auto

    def reset(self):
        self.accumulated_error = 0
        self.last_pv = 0
        self.last_error = 0        

    def _calc_output(self, error, dpv, accum_error):
        Ui = self.oneoveritime * accum_error
        Ud = self.dtime * dpv
        Uk = self.bump + self.pgain * (error + Ui + Ud)

        if Uk > self.auto_max:
            Uk = self.auto_max
        elif Uk < self.auto_min:
            Uk = self.auto_min

        return Uk
    
    def _process_err(self, pv, sp, dt):
        err = sp - pv
        dpv = pv - self.last_pv
        self.last_pv = pv
        accum = self.accumulated_error
        self.accumulated_error += dt * err
        return err, dpv, accum

    def step_output(self, pv, sp, dt=1):
        err, dpv, accum = self._process_err(pv, sp, dt)
        out = self._calc_output(err, dpv, accum)
        self.last_output = out
        return out
    
    def step_output2(self, pv, sp, dt=1):
        err = sp - pv
        dpv = pv - self.last_pv
        ierr = (err + self.last_error) / 2 * dt
        
        if self.dif:
            ierr *= 1 / (1 + err * err / 250)  # Labview's ifactor thingy
        
        # bump (aka controller bias) isn't normally
        # included in Up, but no one else reads my 
        # code :)
        Up = self.bump + self.pgain * err
        Ui = self.oneoveritime * (ierr + self.accumulated_error) * self.pgain
        Ud = self.dtime * dpv * self.pgain
        Uk = Up + Ui + Ud
        
        # XXX debugging
        self.Ui = Ui * self.pgain
        self.Uk = Uk
        self.Ud = Ud * self.pgain
        self.Up = Up
        
        def assert_(val):
            Uk2 = Up + Ud + Ui2
            Uk2 = round(Uk2, 8)
            am = round(val, 8)
            assert Uk2 == am, (Up, Ud, Ui2, Uk2, val)
        
        # Coercion & back calculation      
        back_calc = False
        if Uk > self.auto_max:
            Uk = self.auto_max
            back_calc = True
            am = self.auto_max
        elif Uk < self.auto_min:
            Uk = self.auto_min
            back_calc = True
            am = self.auto_min
            
        if back_calc:
            ierr = self.itime / self.pgain *(am - Up - Ud) - self.accumulated_error
            
        self.accumulated_error += ierr
        self.last_output = Uk
        self.last_pv = pv
        self.last_error = err
        self.last_ierr = ierr
        
        return Uk
        
    step = step_output2

    def __repr__(self):
        return "Output: %.2f Pgain: %.1f Itime: %.2f AccumError: %.4f" % (self.last_output,
                                                                          self.pgain,
                                                                          self.itime,
                                                                          self.accumulated_error)
    __str__ = __repr__
    
def pid_ideal(p, i, d, imax, imin, bump, err, ierr, dpv):
    ui = (1 / i) * ierr
    ud = d * dpv
    uk = bump + p * (err + ui + ud)
    if uk > imax:
        uk = imax
    elif uk < imin:
        uk = imin
    return uk
    

In [300]:
def assert_equal(a,b):
    assert a == b, (a, b)
def test_trap_integration1():
    data = [
        (0, 1, .5),
        (0, 2, 2),
        (0, 3, 4.5)
    ]
    p = PIDController(1,1)
    for pv, sp, op in data:
        p.off_to_auto(pv, sp)
        res_op = p.step(pv, sp)
        assert_equal(p.last_error, sp)
        assert_equal(p.accumulated_error, op)
        assert_equal(res_op, op)
test_trap_integration1()

def check_Ui_backcalc(it, pg, Up, Ud, ierr, accum, val):
    Ui2 = it * (ierr + accum) * pg
    Uk2 = Up + Ud + Ui
    Uk2 = round(Uk2, 8)
    am = round(val, 8)
    assert Uk2 == am, (Up, Ud, Ui, Uk2, val)

In [105]:
def m2s(m):
    return m*60
def s2m(s):
    return s/60
def h2s(h):
    return m2s(h*60)

In [330]:
from collections import deque
class DelayBuffer(deque):
    def __init__(self, delay=30, startvalue=0):
        delay = int(delay)
        self.delay = delay
        super().__init__(startvalue for _ in range(delay + 1))

    def cycle(self, hd):
        self[0] = hd
        self.rotate(1)
        return self[0]

In [104]:
def run(t=3600, interval=0.3):
    cycles = int(t/interval)
    p = PIDController(40, 1, 10, 0, 100)
    s = DOSim(40, 0.1, 0.035, 0.465/0.035)
    sp = 40
    pv = 40
    mg = 0.5
    co2 = 0.7
    co2_flow = co2 * mg
    x = []
    y = []
    t = 0
    for _ in range(cycles):
        op = p.step(sp, pv, interval)
        o2_flow = op * mg
        air_flow = (1-(co2_flow + o2_flow))
        pv = s.step(o2_flow, air_flow, interval)
        x.append(t)
        y.append(pv)
        t += interval
        print(o2_flow, pv, t)
    return x, y
        

In [106]:
from types import FunctionType, MethodType
def printdir(o):
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o, a)
            if not isinstance(v, (FunctionType, MethodType)):
                print(a, v)

In [121]:
p = PIDController(3, m2s(2.28), 0)
steps = 10
p.set_bump(39, 40, 0)
for _ in range(steps):
    op = p.step(39, 40, 1)
printdir(p)

Up: 3 Ui: 0.021929824561403514
Up: 3 Ui: 0.04385964912280703
Up: 3 Ui: 0.06578947368421054
Up: 3 Ui: 0.08771929824561406
Up: 3 Ui: 0.10964912280701755
Up: 3 Ui: 0.13157894736842107
Up: 3 Ui: 0.1535087719298246
Up: 3 Ui: 0.17543859649122812
Up: 3 Ui: 0.1973684210526316
Up: 3 Ui: 0.2192982456140351
AUTO_MODE 1
OFF_MODE 0
accumulated_error 10
auto_max 100
auto_min 0
bump -3
dtime 0
itime 136.79999999999998
last_error 1
last_output 0.2192982456140351
last_pv 39
oneoveritime 0.007309941520467838
pgain 3


In [214]:
def coerce(val, low, high):
    if val > high:
        return high
    elif val < low:
        return low 
    return val

class GasController():
    def __init__(self):
        self.co2_req = 7
        self.o2_max = 500
        
    def calc_o2(self, pc):
        maxo2 = 100-self.co2_req
        o2pc = coerce(pc, 0, maxo2)
        return o2pc * self.o2_max / 100
                

class DOProcess():
    def __init__(self, pv, gain, decay, delay=0):
        self.gain = gain
        self.decay = decay
        self.pv = pv
        self.initial_pv = pv
        self.delay = delay
        self.gdelay = DelayBuffer(self.delay, 0).cycle
        
    def reset(self):
        self.gdelay = DelayBuffer(self.delay, 0).cycle
        self.pv = self.initial_pv
        
    def calc(self, o2flow, dt):
        step_gain = self.gain * dt * o2flow
        step_decay = self.decay * dt
        return step_gain, step_decay
    
    def step(self, o2flow, dt):
        g, d = self.calc(o2flow, dt)
        g = self.gdelay(g)
        self.pv = self.pv + g + d
        return self.pv

In [11]:
from officelib.xllib import *
xl = Excel()
wb = xl.ActiveWorkbook
if wb is None:
    wb = xl.Workbooks.Add()
ws = xl.ActiveSheet
if ws is None:
    ws = wb.Worksheets.Add()
cell_range = ws.Cells.Range

In [242]:
def run2(pid, iters=1000):
    p, i, d = pid
    
    sp = 40
    pv = 39
    op = 0
    dt = 1
    c = PIDController(p, i, d)
    c.set_bump(pv, sp, op)
    proc = DOProcess(pv, .0001, -.0003)
    ctrl = GasController()
    o2 = ctrl.calc_o2(op)
    data = [(pv, op, o2)]
    for i in range(iters):
        pv = proc.step(o2, dt)
        op = c.step(pv, sp, dt)
        o2 = ctrl.calc_o2(op)    
        data.append((pv, op, o2))
    sp = 40
    pv = 39
    op = 0
    dt = 1
    o2 = ctrl.calc_o2(op)
    c.reset()
    proc.reset()
    data = [(pv, op, o2)]
    for i in range(iters):
        pv = proc.step(o2, dt)
        op = c.step(pv, sp, dt)
        o2 = ctrl.calc_o2(op)    
        data.append((pv, op, o2))

    return data

In [326]:
def run3(iters, o2, sp, pid, ctrl, proc, dt):
    data = []; dap = data.append
    abs_ = abs
    s = 0
    for _ in range(iters):
        pv = proc.step(o2, dt)
        op = pid.step(pv, sp, dt)
        o2 = ctrl.calc_o2(op)
        dap((pv, op, abs_(sp-pv)))
    return data
#         s += abs_(pv - sp)
#     return s
    
def run_test(p, i, d, pv=39, sp=40, gain=0.0001, decay=-0.0003, delay=0):
    ctrl = GasController()
    op = 0
    pid = PIDController(p, m2s(i), d)
    pid.set_bump(pv, sp, op)
    o2 = ctrl.calc_o2(op)
    proc = DOProcess(pv, gain, decay, delay)
    data = run3(3600, o2, sp, pid, ctrl, proc, 1)
    return data

def run_test2(p, i, d):
    return run_test(p, i, d, 39, 40, 0.0001, -0.0003)

def abs_integral_sum(data):
    return sum(d[2] for d in data)

def test():
    p = 3
    d = 0
    res = []
    i = 1
    di = 1
    lasts = 99999999999999999999
    while abs(di) > 0.0001:
        d = run_test2(p, i, d, 39, 40, .0001, -.0003)
        s = abs_integral_sum(d)
        res.append((s, (p, i, d)))
        if s > lasts:
            di = -di/2
        i += di
        lasts = s
        #print("\rS: %s I: %s DI: %s           " % (s, i, di), end="")
    print("%d simulations run" % len(res))
    for s, (p, i, d) in sorted(res, key=lambda t:t[0])[:10]:
        print(s, "P: %s I: %s" % (p, i))
    min(res, key=lambda t:t[0])

    
def test2():
    vals = []
    n = 0
    for p in range(1, 30):
        p /= 10
        for i in range(1, 100):
            i /= 10
            d = run_test2(p, i, 0)
            s = abs_integral_sum(d)
            vals.append((p, i, s))
            n += 1
            if not n % 100:
                print("\rRan test #%d" % n, end="")
    vals.sort(key=lambda t: t[2])
    print()
    print("%d simulations run" % len(vals))
    for p, i, s in vals[:10]:
        print(s, "P: %s I: %s" % (p, i))
    return vals
    
def print_res(res):
    for p, i, s in res:
        print(s, "P: %s I: %s" % (p, i))
        
def sort_res(res):
    return sorted(res, key=lambda t: t[0])

def test3():
    p = 2.9
    i = 0.1
    d = 0
    delay=0
    dat = run_test(p, i, d, delay=delay)
    s = abs_integral_sum(dat)
    print(s, "P: %s I: %s" % (p, i))
    

In [282]:
# test()
vals = test2()
#test3()

Ran test #2800
2871 simulations run
1039.1484331098536 P: 2.9 I: 3.4
1039.152705766263 P: 2.9 I: 3.3
1039.2646630827749 P: 2.9 I: 3.5
1039.2875436908234 P: 2.9 I: 3.2
1039.4643229848307 P: 2.9 I: 3.6
1039.5633007865115 P: 2.9 I: 3.1
1039.7000303805062 P: 2.9 I: 3.7
1039.9393504308468 P: 2.9 I: 3.8
1039.988945710738 P: 2.9 I: 3.0
1040.1620144736798 P: 2.9 I: 3.9


In [333]:
from officelib.xllib import *
from officelib.const import xlconst
from pywintypes import com_error

def get_xl():
    xl = Excel()
    wb = xl.Workbooks("PID Model 160921.xlsx")
    ws = wb.Worksheets("PID (2)")
    cr = ws.Cells.Range
    return xl, wb, ws, cr

def plugin_data1(p, i, delay):
    xl, wb, ws, cr = get_xl()
    with screen_lock(xl):
        pr = cr("B3")
        ir = cr("C3")
        dr = cr("Q3")
        pr.Value = p
        ir.Value = i
        dr.Value = delay

def plugin_data2(data):
    """
    :param data: list of tuples (p, i, abs_error)
    """
    xl, wb, ws, cr = get_xl()
    with screen_lock(xl):
        topleft = cr("D34")
        botright = topleft.Offset(len(data), 3)
        botright2 = topleft.End(xlconst.xlDown).Offset(1, 3)
        cr(topleft, botright2).Clear()
        cr(topleft, botright).Value = data

In [325]:
def test4():
    vals = []
    n = 0
    delay=400
    for p in range(1, 20):
        p /= 10
        for i in range(1, 200):
            i /= 10
            d = run_test(p, i, 0, delay=delay)
            s = abs_integral_sum(d)
            vals.append((p, i, s))
            n += 1
            if not n % 100:
                print("\rRan test #%d" % n, end="")
    vals.sort(key=lambda t: t[2])
    print()
    print("%d simulations run" % len(vals))
    for p, i, s in vals[:10]:
        print(s, "P: %s I: %s" % (p, i))
    p, i, _ = vals[0]
    plugin_data(p, i, delay)
    return vals
vals4 = test4()

Ran test #2200
2280 simulations run
2222.7939878347884 P: 1.9 I: 19.0
2222.7948516976426 P: 1.9 I: 18.9
2222.870864644803 P: 1.9 I: 19.1
2222.874045310199 P: 1.9 I: 18.8
2223.0248958976485 P: 1.9 I: 19.2
2223.0321604520896 P: 1.9 I: 18.7
2223.255514673854 P: 1.9 I: 19.3
2223.2698587467758 P: 1.9 I: 18.6
2223.5623823264104 P: 1.9 I: 19.4
2223.5877074329997 P: 1.9 I: 18.5


In [334]:
def sum_error(p, i, delay=0):
    d = run_test(p, i, 0, delay=delay)
    plugin_data1(p, i, delay)
    plugin_data2(d)
    s = abs_integral_sum(d)
    return s

In [336]:
sum_error(2, 3, 10)

1252.439816018017

In [246]:
try:
    pr.Clear()
except NameError:
    pass
data = run2((3,m2s(1),0), 3601)
s = cell_range("D33")
pr = cell_range(s, s.Offset(len(data), 3))
pr.Value = data

decay -0.0003
delay 0
gain 0.0001
initial_pv 39
pv 39
AUTO_MODE 1
OFF_MODE 0
accumulated_error 0
auto_max 100
auto_min 0
bump -3
dtime 0
itime 60
last_error 0
last_output 0
last_pv 0
oneoveritime 0.016666666666666666
pgain 3


In [116]:
from officelib.xllib import *
xl = Excel()

wb = xl.ActiveWorkbook
wb = wb or xl.Workbooks.Add()

ws = xl.ActiveSheet
ws = ws or wb.Worksheets.Add()

cells = ws.Cells
cell_range = ws.Cells.Range

In [224]:
# from decimal import Decimal as D
class TempProcess():
    def __init__(self, delay, env=18.5, g=0.0019254, k=-0.001579031):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.tdelay = DelayBuffer(delay).cycle
        self.g = g / 60
        self.k = k / 60
        self.env = env
        
    def step(self, pv, op):
        op = self.tdelay(op)
        dT = pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        return pv + dpv

In [290]:
def temp_sim(p, i, d, delay, amax, amin, end, op=0, pv=20, sp=37):
    pid = PIDController(p,m2s(i),d,amax,amin, do_ifactor=True)
    pid.man_to_auto(pv, sp, op)
    proc = TempProcess(delay)
    t = 0
    data = [(t, pv, op, 0, 0, 0, 0)]
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t, pv, op, pid.accumulated_error, pid.Uk, pid.Up, pid.Ui))
        if t >= end:
            break
    return data
    

In [301]:
def paste(cells, data):
    cells.Range(cells(1,1), cells(1, 3)).Value = [("T", "PV", "OP")]
    topleft = cells(2,1)
    bottomright = topleft.Offset(len(data), len(data[0]))
    cells.Range(topleft, bottomright).Clear()
    cells.Range(topleft, bottomright).Value = data
    

In [346]:
data = temp_sim(60, 19, 0, m2s(4.7), 100, 0, m2s(600), 0, 21.4)
paste(cells, data)

In [336]:
cell = ws.Cells.Range("H3")
delay = 4
with screen_lock(xl):
    while True:
        data = temp_sim(30, 10, 0, m2s(delay), 100, 0, m2s(600), 0, 21.4)
        paste(cells, data)
        if cell.Value > 38.28:
            break
        delay += .1
delay

4.6999999999999975